**Clone the repository**

In [1]:
! git clone https://github.com/pabloac31/ML-workshop.git
%cd ML-workshop

Cloning into 'ML-workshop'...
remote: Enumerating objects: 22, done.
remote: Total 22 (delta 0), reused 0 (delta 0), pack-reused 22
Unpacking objects: 100% (22/22), done.
/content/ML-workshop


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [4]:
import torch

In [5]:
use_cuda=True
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

CUDA Available:  True
